# 数値解析第４回課題

### 学籍番号：08B22182　氏名：平山聖輝

課題4
$$
\begin{equation}
  \left\{ 
  \begin{alignedat}{3}   
     2x_1 + x_2 - x_3 &= 3 \\   
     x_1 + 3x_2 + x_3 &= 20 \\
    -x_1 + x_2 + 4x_3 &= 33 
  \end{alignedat} 
  \right.
\end{equation}
$$
の解 $ x=(x_1,x_2,x_3)^T $ を
* 掃き出し法
* LU分解を用いる方法
の2通りで求めよ．

連立一次方程式を数値計算的に解く場合，行列 $ A \in R^{n\times n} $ ，$ x \in R^{n\times n} $ ，$ b \in R^{n\times 1} $ を用いて 

$$
Ax = b
$$

とかき，行列 $ A $ と $ b $ を基本変形していくことで解 $ x $ を求めることができるというのは授業でも扱った通りである．今回の課題ではその行列を用いた連立一次方程式の解法の中でも特に有名な**掃き出し法**と**LU分解**の2通りで解を計算してみる． $ A $ と $ b $ は以下の通り．

$$
A = \left[
\begin{array}{ccc}
2 & 1 & -1 \\ 1 & 3 & 1 \\-1 & 1 & 4
\end{array}
\right]
,  
b = \left[
\begin{array}{c}
2 \\ 20 \\ 33
\end{array}
\right]
$$

In [242]:
import numpy as np

mat_A = np.array([
    [ 2.0, 1.0, -1.0],
    [ 1.0, 3.0, 1.0],
    [-1.0, 1.0, 4.0]
])
vec_b = np.array([
    [3.0], 
    [20.0],
    [33.0]
])

### 掃き出し法

以下に掃き出し法の関数を定義する．ただし部分ピボット法により各行で一番大きな絶対値をもつピボットを探し出し，入れ替える行基本変形を施すアルゴリズムになっているため小さな値で割る誤差を防ぐような関数になっている必要があることに注意する．

In [14]:
def gauss_jordan(mat_A, vec_b):
    
    mat_Ab = np.concatenate((mat_A, vec_b), axis = 1) #拡大係数行列
    #np.concatenateで行列の結合，axis=1は横の結合の意味
    print(mat_Ab, "\n↓")
    num_vec_x = mat_A.shape[0] #未知数の数

    #各行でpivot選択して掃き出す
    for pivot in range(num_vec_x):
        #pivot列で成分の絶対値が最大の行を探す
        row_max = pivot
        temp_Ab = mat_Ab.copy() #後でvalue_maxに最も大きいピボットを入れるためにコピーを作っておく
        val_max = temp_Ab[pivot, pivot]
        for row in range(pivot+1, num_vec_x):
            if(val_max < abs(temp_Ab[row,pivot])):
                row_max = row
                val_max = temp_Ab[row,pivot] #これでmat_Abの値をいじることなくvalue_maxを変更できる
        #ピボットの確認
        print("pivot=", val_max)

        #pivotの行と入れ替え
        if (pivot != row_max):
            mat_Ab[pivot,:], mat_Ab[row_max,:] = mat_Ab[row_max,:].copy(), mat_Ab[pivot,:].copy()
            print(mat_Ab, "\n↓")

        #pivot行をpivot=val_maxで割る
        mat_Ab[pivot,:] = mat_Ab[pivot,:] / val_max
        print(mat_Ab, "\n↓")

        #掃き出し操作でpivot行以外の行からpivot行を引く
        for row in range(0,num_vec_x):
            #mat_Ab[pivot,pivot]=1になっているからpivot行をmat_Ab[row,pivot]倍してrow行から引く
            if row != pivot:
                pivot_row = mat_Ab[row, pivot] * mat_Ab[pivot, range(pivot, num_vec_x+1)]
                                                                    #一列拡大してるから↑
                mat_Ab[row, range(pivot, num_vec_x+1)] -= pivot_row
                print(mat_Ab, "\n↓")

    #解の取り出し
    return mat_Ab[:,num_vec_x]


In [250]:
x_GJ = gauss_jordan(mat_A, vec_b)
print("x1=", x_GJ[0], "\nx2=", x_GJ[1], "\nx3=", x_GJ[2])

[[ 2.  1. -1.  3.]
 [ 1.  3.  1. 20.]
 [-1.  1.  4. 33.]] 
↓
pivot= 2.0
[[ 1.   0.5 -0.5  1.5]
 [ 1.   3.   1.  20. ]
 [-1.   1.   4.  33. ]] 
↓
[[ 1.   0.5 -0.5  1.5]
 [ 0.   2.5  1.5 18.5]
 [-1.   1.   4.  33. ]] 
↓
[[ 1.   0.5 -0.5  1.5]
 [ 0.   2.5  1.5 18.5]
 [ 0.   1.5  3.5 34.5]] 
↓
pivot= 2.5
[[ 1.   0.5 -0.5  1.5]
 [ 0.   1.   0.6  7.4]
 [ 0.   1.5  3.5 34.5]] 
↓
[[ 1.   0.  -0.8 -2.2]
 [ 0.   1.   0.6  7.4]
 [ 0.   1.5  3.5 34.5]] 
↓
[[ 1.   0.  -0.8 -2.2]
 [ 0.   1.   0.6  7.4]
 [ 0.   0.   2.6 23.4]] 
↓
pivot= 2.6
[[ 1.   0.  -0.8 -2.2]
 [ 0.   1.   0.6  7.4]
 [ 0.   0.   1.   9. ]] 
↓
[[1.  0.  0.  5. ]
 [0.  1.  0.6 7.4]
 [0.  0.  1.  9. ]] 
↓
[[1. 0. 0. 5.]
 [0. 1. 0. 2.]
 [0. 0. 1. 9.]] 
↓
x1= 5.0 
x2= 2.000000000000001 
x3= 9.0


よって掃き出し法により求められた $ x $ は

$$
x_{GJ}=\left[
\begin{array}{c}
5.0 \\ 2.0 \\ 9.0
\end{array}
\right]
$$

### LU分解による方法

今回の課題で扱うLU分解は行列 $ L $ の対角成分が1になるように分解する方法を採用する．講義中では異なる方法が紹介されていたので注意して関数を作成する．

In [206]:
def LU_division(mat_A, vec_b):
    num_vec_x = mat_A.shape[0] #行数の取得
    mat_L = np.eye(num_vec_x) #L=Aと同じn×nの単位行列
    mat_U = mat_A.copy() #はじめはU=A
    mat_LU = np.concatenate((mat_L, mat_U), axis = 1)
    print(mat_LU, "\n↓")
    

    #A=LUのLU分解を行う
    for pivot in range(num_vec_x):
        temp_U = mat_U.copy() #mat_Uの情報をいじっちゃわないように逃げのtemp_Uを用意
        val = temp_U[pivot, pivot]
        #ピボットの確認
        print("pivot =", val)

        for row in range(pivot+1, num_vec_x):
            #pivot以降の行からpivot行をmat_U[row,pivot]/val倍して引く->U
            pivot_row = (mat_U[row, pivot] / val) * mat_U[pivot, range(pivot, num_vec_x)]
            mat_U[row, range(pivot, num_vec_x)] -= pivot_row
            
            #L[row, pivot]にmat_U[row,pivot]/valを代入
            mat_L[row, pivot] = temp_U[row, pivot] / val

            #LUを表示
            mat_LU = np.concatenate((mat_L, mat_U), axis = 1)
            print(mat_LU, "\n↓")
            
    return mat_L, mat_U

In [244]:
def forward_elimination(mat_L, vec_b):
    #Ly=bの前進消去を行う
    vec_y = np.zeros(vec_b.shape) #yを1×nの行列に
    num_vec_x = mat_L.shape[0]

    for row in range(0,num_vec_x): #前進0～n-1
        y_temp = vec_b[row, 0] #内積の計算をするために逃げの変数用意
        for column in range(0, row, 1):
            y_temp = y_temp - mat_L[row, column]*vec_y[column, 0] #columnとcolumnで縮約！
        vec_y[row, 0] = y_temp / mat_L[row, row] #最後y[row, 0]の係数L[row, row]で割る
        print("y=", vec_y, "\n↓")
        
    return vec_y

In [246]:
def back_substitution(mat_U, vec_y):
    #Ux=yの後退代入を行う
    vec_x = np.zeros(vec_y.shape)
    num_vec_x = mat_U.shape[0]

    for row in range(num_vec_x-1, -1, -1): #後退n-1~0
        x_temp = vec_y[row] #前進消去と考え方は同じ,forの向きが変わるだけ
        for column in range(num_vec_x-1, row, -1):
            x_temp = x_temp - mat_U[row, column]*vec_x[column,0]
        vec_x[row] = x_temp / mat_U[row, row]
        print("x=", vec_x, "\n↓")
    
    return vec_x

In [248]:
mat_L, mat_U = LU_division(mat_A, vec_b)
vec_y = forward_elimination(mat_L, vec_b)
x_LU = back_substitution(mat_U, vec_y)
print("x1=", x_LU[0], "\nx2=", x_LU[1], "\nx3=", x_LU[2])

[[ 1.  0.  0.  2.  1. -1.]
 [ 0.  1.  0.  1.  3.  1.]
 [ 0.  0.  1. -1.  1.  4.]] 
↓
pivot = 2.0
[[ 1.   0.   0.   2.   1.  -1. ]
 [ 0.5  1.   0.   0.   2.5  1.5]
 [ 0.   0.   1.  -1.   1.   4. ]] 
↓
[[ 1.   0.   0.   2.   1.  -1. ]
 [ 0.5  1.   0.   0.   2.5  1.5]
 [-0.5  0.   1.   0.   1.5  3.5]] 
↓
pivot = 2.5
[[ 1.   0.   0.   2.   1.  -1. ]
 [ 0.5  1.   0.   0.   2.5  1.5]
 [-0.5  0.6  1.   0.   0.   2.6]] 
↓
pivot = 2.6
y= [[3.]
 [0.]
 [0.]] 
↓
y= [[ 3. ]
 [18.5]
 [ 0. ]] 
↓
y= [[ 3. ]
 [18.5]
 [23.4]] 
↓
x= [[0.]
 [0.]
 [9.]] 
↓
x= [[0.]
 [2.]
 [9.]] 
↓
x= [[5.]
 [2.]
 [9.]] 
↓
x1= [5.] 
x2= [2.] 
x3= [9.]


よってLU分解を用いた解 $ x $ は

$$
x_{LU}=\left[
\begin{array}{c}
5.0 \\ 2.0 \\ 9.0
\end{array}
\right]
$$

### solver利用

最後にnp.solveを用いて検算する

In [267]:
x_solver = np.linalg.solve(mat_A, vec_b)
print(x_solver)

[[5.]
 [2.]
 [9.]]
